In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

2025-07-31 14:02:18.199270: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753970538.474064      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753970538.560600      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
train_dir = '/kaggle/input/reduced-1/Train'
val_dir = '/kaggle/input/reduced-1/Validation'
test_dir = '/kaggle/input/reduced-1/Test'


In [11]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   horizontal_flip=True,
                                   zoom_range=0.2)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


In [12]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(299, 299),
                                                    batch_size=32,
                                                    class_mode='binary')

val_generator = val_datagen.flow_from_directory(val_dir,
                                                target_size=(299, 299),
                                                batch_size=32,
                                                class_mode='binary')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(299, 299),
                                                  batch_size=32,
                                                  class_mode='binary',
                                                  shuffle=False)

Found 15000 images belonging to 2 classes.
Found 6000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


In [13]:
# --- Load InceptionV3 base model ---
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# --- Add custom classification head with dropout ---
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)                       # Dropout to reduce overfitting
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

I0000 00:00:1753970569.898496      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [14]:
# Phase 1: Transfer Learning
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

print("\n--- Training Top Layers (Transfer Learning) ---")
history_transfer = model.fit(train_generator,
                             epochs=10,  # You can do 10 here
                             validation_data=val_generator)



--- Training Top Layers (Transfer Learning) ---


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


I0000 00:00:1753970588.780566     126 service.cc:148] XLA service 0x7f938045d240 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753970588.781749     126 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753970590.776449     126 cuda_dnn.cc:529] Loaded cuDNN version 90300


  2/469 ━━━━━━━━━━━━━━━━━━━━ 36s 78ms/step - accuracy: 0.4531 - loss: 0.8287   

I0000 00:00:1753970599.239205     126 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


469/469 ━━━━━━━━━━━━━━━━━━━━ 392s 791ms/step - accuracy: 0.5793 - loss: 0.6898 - val_accuracy: 0.7108 - val_loss: 0.5605
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 311s 663ms/step - accuracy: 0.6807 - loss: 0.5939 - val_accuracy: 0.7462 - val_loss: 0.5144
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 308s 656ms/step - accuracy: 0.7079 - loss: 0.5673 - val_accuracy: 0.7492 - val_loss: 0.5108
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 307s 654ms/step - accuracy: 0.7147 - loss: 0.5541 - val_accuracy: 0.7573 - val_loss: 0.4988
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 309s 659ms/step - accuracy: 0.7163 - loss: 0.5525 - val_accuracy: 0.7608 - val_loss: 0.4957
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 325s 693ms/step - accuracy: 0.7267 - loss: 0.5445 - val_accuracy: 0.7630 - val_loss: 0.4898
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 323s 689ms/step - accuracy: 0.7257 - loss: 0.5409 - val_accuracy: 0.7635 - val_loss: 0.4868
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 317s 675ms/step - accuracy: 0.7259 - loss: 0.53

In [ ]:
# --- Fine-Tuning Phase: unfreeze top 50 layers ---
for layer in base_model.layers[-50:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

print("\n--- Fine-Tuning Last 50 Layers ---")
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[early_stop]
)



--- Fine-Tuning Last 50 Layers ---
Epoch 1/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 353s 697ms/step - accuracy: 0.7218 - loss: 0.5451 - val_accuracy: 0.7940 - val_loss: 0.4427
Epoch 2/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 328s 699ms/step - accuracy: 0.7987 - loss: 0.4318 - val_accuracy: 0.8227 - val_loss: 0.3913
Epoch 3/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 322s 687ms/step - accuracy: 0.8395 - loss: 0.3668 - val_accuracy: 0.8433 - val_loss: 0.3557
Epoch 4/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 321s 684ms/step - accuracy: 0.8744 - loss: 0.2982 - val_accuracy: 0.8568 - val_loss: 0.3269
Epoch 5/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 324s 691ms/step - accuracy: 0.8924 - loss: 0.2650 - val_accuracy: 0.8670 - val_loss: 0.3099
Epoch 6/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 330s 705ms/step - accuracy: 0.9012 - loss: 0.2417 - val_accuracy: 0.8690 - val_loss: 0.3023
Epoch 7/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 309s 658ms/step - accuracy: 0.9091 - loss: 0.2220 - val_accuracy: 0.8733 - val_loss: 0.3009
Epoch 8/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 

In [18]:
#%%
# --- Combine Training History ---
def combine_history(h1, h2):
    history = {}
    for key in h1.history:
        history[key] = h1.history[key] + h2.history[key]
    return history
history_final = combine_history(history_transfer, history)

In [19]:
# --- Evaluate Final Model ---
train_loss, train_acc = model.evaluate(train_generator, verbose=0)
val_loss, val_acc = model.evaluate(val_generator, verbose=0)
test_loss, test_acc = model.evaluate(test_generator, verbose=0)

# --- Print Results ---
print("\n--- Final Accuracy & Loss ---")
print(f"Train Accuracy:      {train_acc:.4f} | Loss: {train_loss:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}   | Loss: {val_loss:.4f}")
print(f"Test Accuracy:       {test_acc:.4f}   | Loss: {test_loss:.4f}")


--- Final Accuracy & Loss ---
Train Accuracy:      0.9599 | Loss: 0.1099
Validation Accuracy: 0.8808   | Loss: 0.2910
Test Accuracy:       0.8117   | Loss: 0.4931
